In [1]:
# load env ------------------------------------------------------------------------
import os
import utils

utils.load_env()
os.environ['LANGCHAIN_TRACING_V2'] = "false"


# debug ------------------------------------------------------------------
from langchain.globals import set_debug, set_verbose
set_verbose(True)
set_debug(False)

from langchain_core.messages import (
    AIMessage, 
    HumanMessage,
    ToolMessage
)
from langgraph.graph import END, StateGraph, START
from agents import(
    AgentState,
    agents_metadata,
    agent_names
)
from tools import get_tools_output, all_tools
from chat_history import save_chat_history, load_chat_history
from langgraph.checkpoint.memory import MemorySaver

## Define Tool Node
from langgraph.prebuilt import ToolNode
from typing import Literal

tool_node = ToolNode(all_tools)

def router(state) -> Literal["call_tool", "continue", "supervisor", "creative_communication_agent", "crm_agent", "__end__"]:
    # This is the router
    messages = state["messages"]
    last_message = messages[-1]
    if "FINALANSWER" in last_message.content:
        return "__end__"
    if "supervisor" in last_message.content:
        return "supervisor"
    if "creative_communication_agent" in last_message.content:
        return "creative_communication_agent"
    if "crm_agent" in last_message.content:
        return "crm_agent"
    if last_message.tool_calls:
        # The previous agent is invoking a tool
        return "call_tool"
    else:
        return "continue"


## Workflow Graph ------------------------------------------------------------------------
workflow = StateGraph(AgentState)

# add agent nodes
for name, value in agents_metadata.items():
    workflow.add_node(name, value['node'])
    
workflow.add_node("call_tool", tool_node)

workflow.add_conditional_edges(
    "supervisor",
    router,
    {
        "crm_agent":"crm_agent",
        "creative_communication_agent":"creative_communication_agent",
        "call_tool": "call_tool",
        "__end__": END,
        "continue": END, 
        }
)

workflow.add_conditional_edges(
    "creative_communication_agent",
    router,
    {
        "call_tool": "call_tool",
        "continue": "supervisor", 
        }
)

workflow.add_conditional_edges(
    "crm_agent",
    router,
    {
        "call_tool": "call_tool",
        "continue": "supervisor", 
        }
)

workflow.add_conditional_edges(
    "call_tool",
    # Each agent node updates the 'sender' field
    # the tool calling node does not, meaning
    # this edge will route back to the original agent
    # who invoked the tool
    lambda x: x["sender"],
    {name:name for name in agent_names},
)

workflow.add_edge(START, "supervisor")
graph = workflow.compile()

def submitUserMessage(
    user_input:str, 
    user_id:str="test", 
    keep_chat_history:bool=False, 
    return_reference:bool=False, 
    verbose:bool=False,
    recursion_limit:int=20
    ) -> str:
    
    os.environ['CURRENT_USER_ID'] = user_id
    
    chat_history = load_chat_history(user_id=user_id) if keep_chat_history else []
    chat_history = chat_history[-20:]
    
    # memory only keep chat history only along agents.
    internal_level_memory = MemorySaver()
    graph = workflow.compile(checkpointer=internal_level_memory)

    events = graph.stream(
        {
            "messages": [
                HumanMessage(
                    user_input
                )
            ],
            "chat_history": chat_history
        },
        # Maximum number of steps to take in the graph
        {"recursion_limit": recursion_limit, "thread_id":"a"},
    )
    
    if not verbose:
        events = [e for e in events]
        response = list(events[-1].values())[0]
    else:
        for e in events:
            a = list(e.items())[0]
            a[1]['messages'][0].pretty_print()
        
        response = a[1]
    
    response = response["messages"][0].content
    response = utils.format_bot_response(response, markdown=True)
    
    if keep_chat_history:
        save_chat_history(bot_message=response, human_message=user_input, user_id=user_id)
    
    if return_reference:
        return response, get_tools_output()
    else:
        return response

In [2]:
# from IPython.display import Image, display

# try:
#     display(Image(graph.get_graph(xray=True).draw_mermaid_png()))
# except Exception:
#     # This requires some extra dependencies and is optional
#     pass

In [5]:
# import utils
result = submitUserMessage("ฉันชอบ iphone", keep_chat_history=True, return_reference=True, verbose=True)
utils.notify(sound_effect="purr")

================================== Ai Message ==================================
Name: supervisor

ฉันจะส่งข้อมูลนี้ไปยัง CRM Agent เพื่อดึงข้อมูลลูกค้าเพิ่มเติมเกี่ยวกับความชอบของคุณค่ะ

ส่งข้อความไปยัง CRM Agent:

"crm_agent: สวัสดีค่ะ ฉันได้รับข้อมูลจากลูกค้าว่าพวกเขาชอบ iPhone กรุณาช่วยดึงข้อมูลเพิ่มเติมเกี่ยวกับลูกค้าคนนี้เพื่อให้เราสามารถให้บริการที่ตรงตามความต้องการของพวกเขาได้ค่ะ ขอบคุณค่ะ!"
================================== Ai Message ==================================
Name: crm_agent
Tool Calls:
  get_customer_information (call_A54HDU0TSA0tH3zO98htuVIa)
 Call ID: call_A54HDU0TSA0tH3zO98htuVIa
  Args:
================================= Tool Message =================================
Name: get_customer_information

{'_id': ObjectId('66e7cbc3d82ebda0ef0e499d'), 'user_id': 'test', 'name': 'Jeff', 'age': 93, 'gender': 'Male', 'hobbies_interests': 'ปีนเขา', 'preferred_brands': '', 'preferred_products_categories': 'สินค้าเกี่ยวกับการออกกำลังกาย', 'price_sensitivity': ''}
============

In [6]:
submitUserMessage("ช่่วยแนะนำสินค้าหน่่อย", keep_chat_history=True, return_reference=True, verbose=True)

================================== Ai Message ==================================
Name: supervisor

ฉันจะส่งข้อความนี้ไปยัง CRM Agent เพื่อให้พวกเขาช่วยดึงข้อมูลเกี่ยวกับสินค้าที่เกี่ยวข้องกับความสนใจของคุณในการปีนเขาและ iPhone ค่ะ

ส่งข้อความไปยัง CRM Agent:

"crm_agent: สวัสดีค่ะ! ลูกค้าชื่อ Jeff ได้แสดงความสนใจใน iPhone และการปีนเขา กรุณาช่วยดึงข้อมูลเกี่ยวกับสินค้าที่เกี่ยวข้องกับความสนใจเหล่านี้ เพื่อให้ฉันสามารถแนะนำสินค้าให้กับเขาได้ค่ะ ขอบคุณค่ะ!"
================================== Ai Message ==================================
Name: crm_agent
Tool Calls:
  search_retail_store (call_uuwCj6T4A27KQ1SQGCmx3Vkv)
 Call ID: call_uuwCj6T4A27KQ1SQGCmx3Vkv
  Args:
    query: iPhone
  search_retail_store (call_uKPvHVMsGu0ylnT00hb48cXR)
 Call ID: call_uKPvHVMsGu0ylnT00hb48cXR
  Args:
    query: climbing gear
================================= Tool Message =================================
Name: search_retail_store

[{'_id': 3, 'name': 'Central Mall', 'branch_name': 'Central Pinklao', 'locati

('ส่งข้อความสุดท้ายไปยังคุณ Jeff:\n\n---\n\n**สวัสดีค่ะ Jeff!**\n\nเราทราบว่าคุณมีความสนใจใน iPhone และการปีนเขา เราจึงขอแนะนำสินค้าที่อาจจะตรงกับความต้องการของคุณ:\n\n### สินค้าเกี่ยวกับ iPhone:\n1. **Smartphone Pro**\n   - **ราคา:** 29,999.99 บาท\n   - **รายละเอียด:** สมาร์ทโฟนเรือธงที่มาพร้อมฟีเจอร์ล้ำสมัย\n   - **แบรนด์:** TechCorp\n   - **รุ่น:** ProMax\n   - **การรับประกัน:** 2 ปี\n\n2. **Bluetooth Speaker**\n   - **ราคา:** 10.5 บาท\n   - **รายละเอียด:** ลำโพงบลูทูธพกพาที่มีเสียงเบสลึกและเสียงชัดเจน\n   - **แบรนด์:** SoundWave\n   - **รุ่น:** MiniBass\n   - **การรับประกัน:** 1 ปี\n\n### สินค้าเกี่ยวกับการปีนเขา:\n1. **Yoga Mat**\n   - **ราคา:** 540 บาท\n   - **รายละเอียด:** เสื่อโยคะที่เป็นมิตรกับสิ่งแวดล้อม ไม่ลื่นสำหรับการฝึกซ้อมที่สะดวกสบาย\n   - **แบรนด์:** FlexFit\n   - **รุ่น:** EcoMat 200\n   - **การรับประกัน:** 1 ปี\n\n### ข้อมูลร้านค้า:\n- **ชื่อร้าน:** Central Mall\n- **สาขา:** Central Pinklao\n- **ที่อยู่:** 7/222 Borommaratchachonnani Rd, Arun Amarin, Bangkok 10700\n-